In [22]:
import requests
from pprint import pprint
import base64
import os
import re
import random
from pprint import pprint
import IPython

Before you can use the spotify api, you need to create a developers account at spotify.
[Make a spotify app here](https://developer.spotify.com/my-applications/#!/applications)

They will give your application two tokens. A client id and a secret token. Those should be passed as a string to the two parameters below.

In [82]:
# The tokens needed for the spotify web api.
_id = "118f1dbd33134159aa1089eeed003e4d"
_secret = "dc823e2f09034f00aa22d70b5cf42c7a"

# The root directory to save your music previews.
save_loc = "E:/dataset"

# The desired genres

genres = ["soul", "jazz", "rnb", "rock", "edm_dance", "hiphop", "classical", "funk"]

In [52]:
class Connection:
    """
    Class' object instantiates a connection with spotify. When the connection is alive, queries are made with the query_get
    method.
    """
    def __init__(self, client_id, secret):
        # First header and parameters needed to require an access token.
        param = {"grant_type": "client_credentials"}
        header = {"Authorization": "Basic {}".format(
            base64.b64encode("{}:{}".format(client_id, secret).encode("ascii")).decode("ascii")),
                  'Content-Type': 'application/x-www-form-urlencoded'}
        self.token = requests.post("https://accounts.spotify.com/api/token", param, headers=header).json()["access_token"]
        self.header = {"Authorization": "Bearer {}".format(self.token)}
        self.base_url = "https://api.spotify.com"

    def query_get(self, query, params=None):
        """
        
        :param query: (str) URL coming after example.com
        :param params: (dict)
        :return: (json) 
        """
        r = requests.get(self.base_url + query, params, headers=self.header)
        print(r)
        return requests.get(self.base_url + query, params, headers=self.header).json()


In [92]:
conn = Connection(_id, _secret)
query = ["bill+withers", "marvin+gaye", "aretha+franklin", "stevie+wonder", "ray+charles",
        "otis+redding", "curtis+mayfield"]
genres = ["soul"]

c = 0
for q in query:
    r = conn.query_get("/v1/search/", params={ "q":q, "type":"track", "limit": 50} )
    for item in r["tracks"]["items"]:
        url = item.get("preview_url")
        if url:
            name = item["name"]
            genre_dir = os.path.join(save_loc, "mp3", genres[0])
            os.makedirs(genre_dir, exist_ok=True)
            f = os.path.join(genre_dir, "{}-{}.mp3".format(q.replace("+", "_"), name.replace("?", "").replace("/", "")))

            if not os.path.isfile(f):
                r = requests.get(url)
                print("Saving:", f)
                try:
                    with open(f, "wb") as f:
                        f.write(r.content)
                except OSError:
                    pass
            else:
                print("file already exists")
        else:
            print("could not find url for {}-{}.mp3".format(q, name))
#     c += 1

<Response [200]>
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
fil

In [ ]:
conn = Connection(_id, _secret)

for genre in genres:

    # get genres playlists
    r = conn.query_get('/v1/browse/categories/{}/playlists'.format(genre), {"limit": 50, "offset": 0})
    playlist_ids = [item["id"] for item in r["playlists"]["items"]]
    n = 500
    for i in range(len(playlist_ids)):
        print("running id", _id)
        next_ = True
        offset = 0
        c = 0
        
        while next_ is not None and c < 10:  # count is limited because it will iterate over the same playlists
            print("Next query of the same playlist", next_)

            r = conn.query_get('/v1/users/spotify/playlists/' + playlist_ids[i], {"offset": offset, "limit": n})
            try:
                next_ = r["tracks"]["next"]
                tracks = r["tracks"]["items"]
            except KeyError:
                pprint(r)
            for j in range(len(tracks)):
                name = tracks[j]["track"]["name"]
                name = re.sub("\W", "_", name)
                artist = tracks[j]["track"]["artists"][0]["name"]
                artist = re.sub("\W", "_", artist)
                url = tracks[j]["track"]["preview_url"]
                if url:
                    genre_dir = os.path.join(save_loc, "mp3", genre)
                    os.makedirs(genre_dir, exist_ok=True)
                    f = os.path.join(genre_dir, "{}-{}.mp3".format(artist, name))

                    if not os.path.isfile(f):
                        r = requests.get(url)
                        print("Saving {}-{}.mp3".format(artist, name))
                        with open(f, "wb") as f:
                            f.write(r.content)
                    else:
                        print("file already exists")
                else:
                    print("could not find url for {}-{}.mp3".format(artist, name))
            offset += n
            c += 1

# 30 seconds disco!

When you have ran above code snippet above, you can enjoy your 30 seconds disco.

![](https://media.giphy.com/media/7IKOyJ6j9fCUw/giphy.gif)

In [5]:
genres_d = os.listdir(save_loc)
for _ in range(5):
    genre_current = genres_d[random.randint(0, len(genres_d) - 1)]
    nrs = os.listdir(os.path.join(save_loc, genre_current))
    choice = random.randint(0, len(nrs) - 1)
    print(genre_current)
    a = IPython.display.Audio(os.path.join(save_loc, genre_current, nrs[choice]))
    IPython.display.display(a)


reggae


funk


reggae


classical


reggae
